In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

import  matplotlib.pyplot  as plt
import gensim
import numpy as np
import spacy
from rcare.nlp.customize_nlp import rcare
rcare = rcare(spacy.load('en_core_web_sm'))
nlp = rcare.nlp

from rcare.readutils import DataFrameUtils as Utils

from rcare.nlp.sections_rcare import get_sections

from rcare.es_helper import ESHelper
ES_CONFIG = {
    "host": [
        {
            "host": "localhost",                  
            "port":"9200"
        }
    ],
    "log": "error"
};

esHelper = ESHelper(ES_CONFIG)

from rcare.sqlutils import SqlUtils
sqlUtils = SqlUtils("/home/paperspace/dev/sqllite-node/app/data/training.db")

import pandas as pd
import json

2018-09-10 11:56:56,257 : INFO : 'pattern' package not found; tag filters are not available for English
2018-09-10 11:56:58,557 : INFO : ['tagger', 'parser', 'ner', 'CardinalRecognizer']


In [3]:
#nlp = spacy.load("en_core_web_sm")


DATA_PATHS = [
    #'data/lexpredict-contraxsuite-samples/agreements/construction/'
    #'data/lexpredict-contraxsuite-samples/agreements/citi/'
    "/home/paperspace/dev/en-nlp/data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/"
]
#DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/'
#DATA_PATHS.append(DATA_PATH)
#df_2011_q3 = Utils.read_data(path = DATA_PATH)

#DATA_PATH = 'data/lexpredict-contraxsuite-samples/credit_card_agreements/citi/'
#DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/'
#df_2011_q4 = Utils.read_data(path = DATA_PATH)



#DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2012_Q4/text/'
#DATA_PATHSdf_2012_q4 = Utils.read_data(path = DATA_PATH)
#DATA_PATHS.append(DATA_PATH)

#DATA_PATHS
pd_txtFileNames = sqlUtils.select("select txtFileName from meta_data where useForTraining = 1 \
and agreementFileName like 'creditcardagreement_%' and lineCount > 200 \
order by agreementFileName")
pd_txtFileNames

,txtFileName
0,creditcardagreement_1000.TXT
1,creditcardagreement_1045.TXT
2,creditcardagreement_1046.TXT
3,creditcardagreement_1047.TXT
4,creditcardagreement_105.TXT
5,creditcardagreement_1061.TXT
6,creditcardagreement_117.TXT
7,creditcardagreement_118.TXT
8,creditcardagreement_1184.TXT
9,creditcardagreement_119.TXT


In [4]:
#sqlUtils.test()
fileNames = list(Utils.flatten(pd_txtFileNames[:30].values )) #+ list(Utils.flatten( pd_txtFileNames[100:110].values))\
#+ list(Utils.flatten( pd_txtFileNames[190:].values))
fileNames

['creditcardagreement_1000.TXT',
 'creditcardagreement_1045.TXT',
 'creditcardagreement_1046.TXT',
 'creditcardagreement_1047.TXT',
 'creditcardagreement_105.TXT',
 'creditcardagreement_1061.TXT',
 'creditcardagreement_117.TXT',
 'creditcardagreement_118.TXT',
 'creditcardagreement_1184.TXT',
 'creditcardagreement_119.TXT',
 'creditcardagreement_120.TXT',
 'creditcardagreement_121.TXT',
 'creditcardagreement_1271.TXT',
 'creditcardagreement_1286.TXT',
 'creditcardagreement_1309.TXT',
 'creditcardagreement_1366.TXT',
 'creditcardagreement_1368.TXT',
 'creditcardagreement_1369.TXT',
 'creditcardagreement_1405.TXT',
 'creditcardagreement_1421.TXT',
 'creditcardagreement_1422.TXT',
 'creditcardagreement_1444.TXT',
 'creditcardagreement_1452.TXT',
 'creditcardagreement_1453.TXT',
 'creditcardagreement_1560.TXT',
 'creditcardagreement_1561.TXT',
 'creditcardagreement_1581.TXT',
 'creditcardagreement_1612.TXT',
 'creditcardagreement_1613.TXT',
 'creditcardagreement_1627.TXT']

In [5]:

EntityDocument = namedtuple('EntityDocument', "text startChar endChar label")
SentenceDocument = namedtuple('SentenceDocument', 'words tags names index name sectionId sentId sentText startChar \
                                                    endChar sectionText similarity')

In [17]:
alldocs , tokens = [], []
index = 0
for path in DATA_PATHS:
    #df_g = Utils.read_data_gen(path = path, numfiles = 20)    
    df_g = Utils.read_data_gen(filenames = fileNames, path = path, numfiles = None)    
    for file_idx, fileData in enumerate(df_g):    
        #text =  Utils.clean_document(fileData.text, rm_stop_words = True)
        name = fileData.name
        text = fileData.text        
        logging.info("processing {}".format(name))
        #doc = nlp(text)                
        tokens = []           
        for sec_idx, section in enumerate(get_sections(text, score_threshold=0.6, nlp = nlp)):
            tokens = []           
            #section = Utils.clean_document(section, rm_stop_words = True)
            section = nlp(section)
            for sent_idx, sent in enumerate(section.sents):
                tokens = []        
                for w in sent:  
                    if (w.is_digit):
                        #tokens.append("-DIGIT-")  
                        continue;
                    if (w.like_num):
                        #tokens.append("-DIGIT-")  
                        continue;                    
                    elif (w._.is_cardinal):
                        #tokens.append("-CARDINAL-") 
                        continue;
                    elif (w.is_punct):
                        #tokens.append("-PUNCT-")     
                        continue;
                    elif (w.is_stop):
                        #tokens.append("-STOP-")                          
                        continue;
                    elif (w.like_url):
                        #tokens.append("-URL-")                  
                        continue;
                    elif(w.lemma_ == '-PRON-'):
                         tokens.append(w.lower_)
                    elif (w.text.strip() not in ('\n', '$','\n\n' , '\n\n\n', '\n\n\n\n','' ) ) :                 
                        tokens.append(w.lemma_)  
                        
                if(len(tokens) > 0):
                    tags = [index]
                    names = [name]                    
                    alldocs.append(SentenceDocument(tokens, tags, names, index, name, sec_idx ,sent_idx,
                                                   sent.text, sent.start_char, sent.end_char, sent.doc.text,                                                    
                                                   0.0,
                                                   #list(map(lambda ent: EntityDocument(ent.text, ent.start_char, ent.end_char,  ent.label_), sent.doc.ents))
                                                  ))
                    index = index + 1
        
                
    
        
            
    

2018-09-10 12:16:25,031 : INFO : Reading file creditcardagreement_1000.TXT
2018-09-10 12:16:25,034 : INFO : processing creditcardagreement_1000.TXT
2018-09-10 12:16:27,781 : INFO : 316
2018-09-10 12:16:30,356 : INFO : Reading file creditcardagreement_1045.TXT
2018-09-10 12:16:30,359 : INFO : processing creditcardagreement_1045.TXT
2018-09-10 12:16:33,583 : INFO : 510
2018-09-10 12:16:35,924 : INFO : Reading file creditcardagreement_1046.TXT
2018-09-10 12:16:35,926 : INFO : processing creditcardagreement_1046.TXT
2018-09-10 12:16:38,959 : INFO : 493
2018-09-10 12:16:41,962 : INFO : Reading file creditcardagreement_1047.TXT
2018-09-10 12:16:41,965 : INFO : processing creditcardagreement_1047.TXT
2018-09-10 12:16:45,350 : INFO : 576
2018-09-10 12:16:48,551 : INFO : Reading file creditcardagreement_105.TXT
2018-09-10 12:16:48,554 : INFO : processing creditcardagreement_105.TXT
2018-09-10 12:16:53,066 : INFO : 400
2018-09-10 12:16:55,870 : INFO : Reading file creditcardagreement_1061.TXT
20

In [ ]:
esHelper.bulk_stream_collection( pd.DataFrame( alldocs,columns = SentenceDocument._fields).to_dict(orient='records'), 
                                index = "credit", doc_type = "agreement")

In [10]:
list(esHelper.prep_bulk_iterator(pd.DataFrame( alldocs,columns = SentenceDocument._fields).to_dict(orient='records'), 
                                index = "credit", doc_type = "agreement", get_id = get_id))

[{'_op_type': 'update',
  '_index': 'credit',
  '_type': 'agreement',
  '_id': 'creditcardagreement_1000.TXT_0',
  'doc': {'words': ['mastercard',
    'and',
    'visa',
    'line',
    'of',
    'credit',
    'agreement'],
   'tags': [0],
   'names': ['creditcardagreement_1000.TXT'],
   'index': 0,
   'name': 'creditcardagreement_1000.TXT',
   'sectionId': 0,
   'sentId': 0,
   'sentText': 'MASTERCARD AND VISA LINE OF CREDIT AGREEMENT\n\n',
   'startChar': 0,
   'endChar': 46,
   'sectionText': 'MASTERCARD AND VISA LINE OF CREDIT AGREEMENT\n\nThe purpose of the Agreement is to establish the terms and conditions of\nyour credit card agreement with McCoy Federal Credit Union. In this\nagreement, the words "you" and "your" mean any person receiving and using\nthe card and that by using the card for the first time "you" agree to all terms\nand conditions in this agreement. "Card" means a Mastercard or VISA credit\ncard and any duplicates and renewals the Credit Union issues. "Account"\nme

In [14]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

In [15]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [18]:
vector_size = 200
epochs = 20
window = 5
simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, dbow_words=1, vector_size=vector_size, window = window, negative=5, hs=0, min_count=2, sample=0, 
            epochs=epochs, workers=cores),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, dm_mean=1, vector_size=vector_size, window=window, negative=5, hs=0, min_count=2, sample=0, 
            epochs=epochs, workers=cores),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, vector_size=vector_size, window=window, negative=5, hs=0, min_count=2, sample=0, 
            epochs=epochs, workers=cores),
]
'''
for model in simple_models:
    model.build_vocab(alldocs)
    print("%s vocabulary scanned & state initialized" % model)
'''    
#model = simple_models[1]
#model.epochs
model = simple_models[0]
model.build_vocab(alldocs)

2018-09-10 12:20:52,683 : INFO : using concatenative 2200-dimensional layer1
2018-09-10 12:20:52,685 : INFO : collecting all words and their counts
2018-09-10 12:20:52,686 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-09-10 12:20:52,751 : INFO : collected 2723 word types and 7715 unique tags from a corpus of 7715 examples and 79659 words
2018-09-10 12:20:52,753 : INFO : Loading a fresh vocabulary
2018-09-10 12:20:52,762 : INFO : effective_min_count=2 retains 1962 unique words (72% of original 2723, drops 761)
2018-09-10 12:20:52,764 : INFO : effective_min_count=2 leaves 78898 word corpus (99% of original 79659, drops 761)
2018-09-10 12:20:52,775 : INFO : deleting the raw counts dictionary of 2723 items
2018-09-10 12:20:52,776 : INFO : sample=0 downsamples 0 most-common words
2018-09-10 12:20:52,777 : INFO : downsampling leaves estimated 78898 word corpus (100.0% of prior 78898)
2018-09-10 12:20:52,787 : INFO : estimated required memory for 1962 wo

In [19]:

#model.train(alldocs, total_examples=model.corpus_count, epochs=model.epochs)

max_epochs = 1
alpha = 0.025

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(alldocs, total_examples=model.corpus_count, epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha


2018-09-10 12:21:15,636 : INFO : training model with 8 workers on 1962 vocabulary and 200 features, using sg=1 hs=0 sample=0 negative=5 window=5


iteration 0


2018-09-10 12:21:16,315 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-10 12:21:16,471 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-10 12:21:16,487 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-10 12:21:16,519 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-10 12:21:16,525 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-10 12:21:16,530 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 12:21:16,532 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 12:21:16,536 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 12:21:16,536 : INFO : EPOCH - 1 : training on 79659 raw words (86613 effective words) took 0.9s, 100987 effective words/s
2018-09-10 12:21:16,999 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-10 12:21:17,287 : INFO : worker thread f

2018-09-10 12:21:23,951 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 12:21:23,973 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 12:21:23,981 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 12:21:23,982 : INFO : EPOCH - 10 : training on 79659 raw words (86613 effective words) took 0.8s, 115213 effective words/s
2018-09-10 12:21:24,552 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-10 12:21:24,561 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-10 12:21:24,615 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-10 12:21:24,723 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-10 12:21:24,729 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-10 12:21:24,743 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 12:21:24,750 : INFO : worker thread 

2018-09-10 12:21:31,836 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-10 12:21:31,848 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-10 12:21:31,866 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-10 12:21:31,877 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-10 12:21:31,890 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 12:21:31,909 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 12:21:31,916 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 12:21:31,917 : INFO : EPOCH - 20 : training on 79659 raw words (86613 effective words) took 0.9s, 98486 effective words/s
2018-09-10 12:21:31,919 : INFO : training on a 1593180 raw words (1732260 effective words) took 16.3s, 106389 effective words/s


In [32]:
window_start = 120 # default(72, 13)
window_size = 2
len(alldocs), list(map(lambda tag: tag.sentText, alldocs[window_start:window_start + window_size]))
#alldocs[window_start].sectionText, 

(7715,
 ['RETURNS AND ADJUSTMENTS: Merchants who honor your card will give\nyou credit for returns or adjustments by sending us a credit slip which we\nshall post to your Plan.',
  'If your credits and payments exceed what you owe\nus, we will hold this credit balance against future purchases or refund it to\nyour share account upon your written request (if it is $1.00 or more).'])

In [33]:
#alldocs[doc_id]
print(model.docvecs.count, len(alldocs))
doc = []
#window_start = 50
#window_size = 10
for d in alldocs[window_start:window_start + window_size]:
    doc = doc + d.words 
print (" ".join(doc)  )

7715 7715
return and adjustment merchant honor card credit return adjustment send credit slip shall post plan if credit payment exceed owe hold credit balance future purchase refund share account write request


In [34]:
#model = simple_models[0]#random.choice(simple_models)  # and a random model
#print(model)
def updateSimilarity(sims, taggedDocs)   :
    for index, sim in sims:
        taggedDocs[index] = taggedDocs[index]._replace(similarity = sim)
    return taggedDocs;

inferred_vector = model.infer_vector(doc)
#inferred_vector = model.infer_vector(['finance', 'charges', 'on', 'purchases' ])
sims = model.docvecs.most_similar([inferred_vector], topn=model.docvecs.count)  # get *all* similar documents

print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, idx in [('MOST', 0),  ('MEDIAN', len(sims)/2), ('LEAST', len(sims) - 1)]:
    index = int(idx)
    #print (label, idx, sims[index])
    print(u'%s %s: %s \n«%s»\n' % (label, sims[index], alldocs[sims[index][0]].name ,
                              alldocs[sims[index][0]].sentText))
    

2018-09-10 12:26:22,481 : INFO : precomputing L2-norms of doc weight vectors


SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow+w,d200,n5,w5,mc2,t8):

MOST (7417, 0.7127496004104614): creditcardagreement_1613.TXT 
«If your credits and payments exceed what you owe
us, we will hold this credit balance against future purchases or refund it to
your share account upon your written request (if it is $1.00 or more).»

MEDIAN (2924, 0.2884993851184845): creditcardagreement_121.TXT 
«Periodic Rate and Annual Percentage Rate for Purchase Advances and Cash
Advances.»

LEAST (3870, -0.14242655038833618): creditcardagreement_1309.TXT 
«(L800)»



/home/paperspace/anaconda3/envs/en-nlp/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [35]:
alldocs = updateSimilarity(sims, alldocs)

In [ ]:
doc2vec_pkl_filename = "rcare/nlp/doc2vec.pickle"
#doc2vec_model_pkl = open(mlp_pkl_filename, 'wb')
#pickle.dump(best_classifier, doc2vec_model_pkl)
model.save(doc2vec_pkl_filename)


In [ ]:
similarities = findSimilarities(doc)

In [ ]:
similarities.similarityByDoc['creditcardagreement_1000.TXT'][1]
#simDoc,simDoc.sent.doc

In [ ]:
for simDoc in similarities.similarityByDoc:
    print("===>", simDoc, similarities.similarityByDoc[simDoc][0].similarity , 
          similarities.similarityByDoc[simDoc][0].sent if (len (similarities.similarityByDoc[simDoc][1].doc) > 350)
          else similarities.similarityByDoc[simDoc][0].doc )
    print("*****")
    

In [ ]:

alldocs[0], alldocs[0]._asdict()


In [ ]:
df = pd.DataFrame( similarities.similarityByDoc['creditcardagreement_1000.TXT'],columns = XTaggedDocument._fields)
df.query("sectionIdx == '1'")[['similarity']].describe()

In [ ]:
records = df.to_dict(orient='records')

In [ ]:
actions = ["""{ "index" : { "_index" : "%s"} }\n""" % (index) +json.dumps(records[j])
                    for j in range(len(records))]

In [ ]:
#{"name": "creditcardagreement_1046.TXT", "records": records[0:5]}

def prep_bulk_data(data, id, index = "credit", indexType = "agreements"):
            for item in data:                
                yield {
                       '_index': index,
                       '_type': indexType,
                       '_id': item[id],
                       '_source': item
                }
name =  "creditcardagreement_1000.TXT"
try:            
    esHelper.bulk_stream(prep_bulk_data(data = [{"name": name, 
                                                 "records": pd.DataFrame( similarities.similarityByDoc[name],columns = XTaggedDocument._fields).to_dict(orient='records')
                                                }], id = "name"))        
except Exception as ex :
    print ('Ingnoring error {}'.format(ex));
        

In [ ]:
 similarities.simForDoc['creditcardagreement_1046.TXT']

In [ ]:
#esHelper.info()
esConnection = esHelper.connection()

In [ ]:
#alldocs
df = pd.DataFrame( alldocs,columns = SentenceDocument._fields)

In [ ]:
def prep_bulk_data(data,index = "credit", indexType = "agreements"):
            for item in data:                
                yield {
                       '_index': index,
                       '_type': indexType,                       
                       '_source': item
                }
                
for name in ['creditcardagreement_1000.TXT']:
    print (name);
    try:            
        esHelper.bulk_stream(prep_bulk_data(data =  pd.DataFrame( alldocs,columns = SentenceDocument._fields).to_dict(orient='records')))
    except Exception as ex :
        print ('Ingnoring error {}'.format(ex));

In [ ]:
pd.DataFrame( alldocs,columns = SentenceDocument._fields).query("name != 'creditcardagreement_1000.TXT'")

In [ ]:
tp = list(df.itertuples(index = False, name = "SentenceDocument"))

In [ ]:
tp[0:10]

In [ ]:
df = pd.DataFrame( alldocs,columns = SentenceDocument._fields)

In [ ]:
body = {"query": {"match": {"name":"creditcardagreement_1000.TXT"}}}
#result = esConnection.search(index="credit1", body = body )['hits']

def search(esconnection, index, doc_type = None, body = None):
    try:
        result = esconnection.search(index=index, doc_type = doc_type, body = body )['hits']
        keys = [key for key in result]
        EsResult = namedtuple('EsResult' , keys)
        return EsResult(*[result[key] for key in keys])
    except Exception as err:
        raise err;   
def search_nt(esconnection, index, source_np, doc_type = None, body = None ):
    try:
        result = search(esconnection, index, doc_type = doc_type, body = body)
        for hit in result.hits:
            yield source_np(*[hit['_source'][key] for key in source_np._fields]) 
    except Exception as err:
        raise err;           


In [ ]:
list(search_nt(esConnection, "credit", body = body, source_np = SentenceDocument))

In [ ]:
body = {"query": {"match": {"name":"creditcardagreement_1000.TXT"}}}
list(esHelper.search_nt ("credit", body = body, source_np = SentenceDocument))

In [ ]:
esHelper.bulk_stream_collection( pd.DataFrame( alldocs,columns = SentenceDocument._fields).to_dict(orient='records'), 
                                index = "credit", doc_type = "agreement")

AttributeError: 'str' object has no attribute 'decode'